#**To categorize a user's gender we are going to deploy a classification model .**



#**Importing Necessary Libraries**

In [3]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
from google.colab import drive
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,classification_report, precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score



#**Data Loading**

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
flights = pd.read_csv("/flights.csv")
hotels = pd.read_csv("/hotels.csv")
users = pd.read_csv("/users.csv")

## Dataset First view

In [15]:
flights.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,09/26/2019
1,0,0,Florianopolis (SC),Recife (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,09/30/2019
2,1,0,Brasilia (DF),Florianopolis (SC),firstClass,1487.52,1.66,637.56,CloudFy,10/03/2019
3,1,0,Florianopolis (SC),Brasilia (DF),firstClass,1127.36,1.66,637.56,CloudFy,10/04/2019
4,2,0,Aracaju (SE),Salvador (BH),firstClass,1684.05,2.16,830.86,CloudFy,10/10/2019


In [16]:
hotels.head()

,travelCode,userCode,name,place,days,price,total,date
0,0,0,Hotel A,Florianopolis (SC),4,313.02,1252.08,09/26/2019
1,2,0,Hotel K,Salvador (BH),2,263.41,526.82,10/10/2019
2,7,0,Hotel K,Salvador (BH),3,263.41,790.23,11/14/2019
3,11,0,Hotel K,Salvador (BH),4,263.41,1053.64,12/12/2019
4,13,0,Hotel A,Florianopolis (SC),1,313.02,313.02,12/26/2019


In [17]:
users.head()

,code,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44


#**Data Preprocessing**

### Dataset Rows & Columns count

In [21]:
users.shape

(1340, 5)

### Checking Null Values for User Dataset



In [18]:
#check for missing values
users.isnull().sum()

,0
code,0
company,0
name,0
gender,0
age,0


In [19]:
#check for duplicate rows
users.duplicated().sum()

0

In [20]:
#check for data types of different features
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   code     1340 non-null   int64 
 1   company  1340 non-null   object
 2   name     1340 non-null   object
 3   gender   1340 non-null   object
 4   age      1340 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 52.5+ KB


**Summary:-**
1. The gender feature, our dependent variable, will be encoded using a label encoder to convert its categorical values (i.e., male and female) into numeric representations suitable for the classification model. Here, 1 will denote male, and 0 will denote female.

2. The company feature holds a non-ordinal relationship, meaning the categories do not have a meaningful order. Therefore, using One-Hot Encoding is the best option to transform its categorical values into a numeric format suitable for the model.

3. For the name feature, we will apply an NLP model, such as a sentence transformer, to extract meaningful embeddings. Additionally, we will use PCA (Principal Component Analysis) to reduce dimensionality before using it in the classification model.

In [24]:
# 5 Point Summary of numerical features
users.describe()

,code,age
count,1340.000000,1340.000000
mean,669.500000,42.742537
std,386.968991,12.869779
min,0.000000,21.000000
25%,334.750000,32.000000
50%,669.500000,42.000000
75%,1004.250000,54.000000
max,1339.000000,65.000000


In [25]:
users.describe(include=['object', 'category'])

,company,name,gender
count,1340,1340,1340
unique,5,1338,3
top,4You,Charlotte Johnson,male
freq,453,2,452


In [26]:
users['company'].value_counts()

,count
company,
4You,453
Acme Factory,261
Wonka Company,237
Monsters CYA,195
Umbrella LTDA,194


In [28]:
users.sample(n=5)

,code,company,name,gender,age
987,987,Acme Factory,Chris Dougherty,none,40
866,866,Wonka Company,Yvonne Ahmad,female,46
109,109,4You,Maria Averitt,female,24
753,753,Wonka Company,Eric Capps,male,51
1270,1270,Umbrella LTDA,Christopher Wilkerson,none,47


In [29]:
users[users['name']=='Charlotte Johnson']

,code,company,name,gender,age
316,316,4You,Charlotte Johnson,female,64
496,496,Monsters CYA,Charlotte Johnson,none,51


In [30]:
# Summary of categorical features
users.describe(include=object)

,company,name,gender
count,1340,1340,1340
unique,5,1338,3
top,4You,Charlotte Johnson,male
freq,453,2,452


1. As we see ,we have 3 categories in our target variable ,let's deep dive into the target variable.


In [31]:
users['gender'].value_counts()

,count
gender,
male,452
female,448
none,440


In [9]:
#filtering records based on relavent categories in the target variable
user_df=users[(users['gender']=='male') | (users['gender']=='female') ]

NameError: name 'users' is not defined

In [11]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
one_hot_encoder = OneHotEncoder()

# One-Hot Encode the 'company' feature
company_encoded = one_hot_encoder.fit_transform(user_df[['company']]).toarray()

# Add one-hot encoded columns back to the DataFrame
company_encoded_df = pd.DataFrame(company_encoded, columns=one_hot_encoder.get_feature_names_out(['company']))
user_df = pd.concat([user_df, company_encoded_df], axis=1)

# Encode the 'gender' feature using Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
user_df['gender_encoded'] = label_encoder.fit_transform(user_df['gender'])


NameError: name 'user_df' is not defined

In [ ]:
# # Encode userCode and company to numeric values
# label_encoder = LabelEncoder()

# user_df['company_encoded'] = label_encoder.fit_transform(user_df['company'])
# user_df['gender_encoded'] = label_encoder.fit_transform(user_df['gender'])



In [14]:
user_df.head()

NameError: name 'user_df' is not defined

In [16]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Initialize the SentenceTransformer model
model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v4_MiniLM-L6')

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Handle missing values and ensure all are strings
text_columns = ['name']
for column in text_columns:
    user_df[column] = user_df[column].fillna('').astype(str)  # Replace NaN with empty string

# Encode text columns into embeddings
for column in text_columns:
    user_df[column + '_embedding'] = user_df[column].apply(lambda text: model.encode(text))

# Concatenate embeddings into a single feature vector (if required)
text_embeddings = user_df[[column + '_embedding' for column in text_columns]].values.tolist()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Update numerical features to include one-hot encoded company columns
numerical_features = ['code', 'age'] + [col for col in user_df.columns if col.startswith('company_')]

# Extract numerical features
X_numerical = user_df[numerical_features].values

# Apply PCA to text embeddings
n_components = 23  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
text_columns = ['name']

# Create an empty array to store PCA-transformed embeddings
text_embeddings_pca = np.empty((len(user_df), n_components * len(text_columns)))

for i, column in enumerate(text_columns):
    embeddings = np.array(user_df[column + '_embedding'].tolist())
    embeddings_pca = pca.fit_transform(embeddings)
    text_embeddings_pca[:, i * n_components:(i + 1) * n_components] = embeddings_pca

# Combine PCA-transformed text embeddings and numerical features
X = np.hstack((text_embeddings_pca, X_numerical))

# Target variable
y = user_df['gender_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
len(text_embeddings)

In [ ]:
print("Shape of X:", X.shape)

In [ ]:
user_df[numerical_features].values

In [ ]:
# Calculate missing values in percentage for training and test data
train_missing_percentage = pd.DataFrame(X_train).isnull().sum()/len(pd.DataFrame(X_train))*100
test_missing_percentage = pd.DataFrame(X_test).isnull().sum() * 100

print("Missing values in training data (percentage):\n", train_missing_percentage)
print("Missing values in test data (percentage):\n", test_missing_percentage)


In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # You can also use 'median' or 'constant'
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)


#**Logistic Regression**

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

# Impute missing values
imputer = SimpleImputer(strategy='median')  # You can adjust the strategy if needed
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Initialize and fit the Logistic Regression model
lr_classifier = LogisticRegression(random_state=42)
lr_classifier.fit(X_train, y_train)

# Predictions
y_pred_lr = lr_classifier.predict(X_test)

# Accuracy
accuracy = lr_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

# Classification report
report = classification_report(y_test, y_pred_lr)
print("\nClassification Report:\n", report)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_lr)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


NameError: name 'X_train' is not defined

In [27]:
from sklearn.impute import SimpleImputer

# Create an imputer to fill missing values with the median
imputer = SimpleImputer(strategy='median')

# Apply the imputer to both training and validation data
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

# Now train the model on the imputed data
lr_classifier.fit(X_train_imputed, y_train)

# Predictions on training and validation sets
y_train_pred = lr_classifier.predict(X_train_imputed)
y_val_pred = lr_classifier.predict(X_val_imputed)

# Calculate accuracy on training and validation sets
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the accuracies
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

# Check if overfitting
if train_accuracy > val_accuracy:
    print("The model is overfitting.")
else:
    print("The model is not overfitting.")


NameError: name 'X_train' is not defined

In [29]:
model_result = pd.DataFrame([['Logistic Regression Baseline', train_accuracy,val_accuracy]],
               columns = ['Model', 'Train accuracy', 'Validation accuracy'])

model_result

NameError: name 'train_accuracy' is not defined

#**Decision Tree Classifier**

In [32]:
# Initialize a Random Forest Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Fit the model to the training data
dt_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred_dt = dt_classifier.predict(X_test)

# Calculate and print accuracy
accuracy = dt_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

# Generate a classification report
report = classification_report(y_test, y_pred_dt)
print("\nClassification Report:\n", report)

cm = confusion_matrix(y_test, y_pred_dt)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()



NameError: name 'DecisionTreeClassifier' is not defined

In [34]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model on the training set
dt_classifier.fit(X_train, y_train)

# Predictions on training and validation sets
y_train_pred = dt_classifier.predict(X_train)
y_val_pred = dt_classifier.predict(X_val)

# Calculate accuracy on training and validation sets
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the accuracies
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

# Check if overfitting
if train_accuracy > val_accuracy:
    print("The model is overfitting.")
else:
    print("The model is not overfitting.")

NameError: name 'train_test_split' is not defined

In [36]:
model = pd.DataFrame([['Decesion Tree Classifier Baseline', train_accuracy,val_accuracy]],
               columns = ['Model', 'Train accuracy', 'Validation accuracy'])
model_result = pd.concat([model_result,model],axis=0,ignore_index = True)
model_result

NameError: name 'train_accuracy' is not defined

#**Random Forest Classifier**

In [39]:
# Initialize a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred_rf = rf_classifier.predict(X_test)

NameError: name 'RandomForestClassifier' is not defined

###**Evaluation**

In [42]:
# Calculate and print accuracy
accuracy = rf_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

# Generate a classification report
report = classification_report(y_test, y_pred_rf)
print("\nClassification Report:\n", report)

cm = confusion_matrix(y_test, y_pred_rf)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()



NameError: name 'rf_classifier' is not defined

In [44]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model on the training set
rf_classifier.fit(X_train, y_train)

# Predictions on training and validation sets
y_train_pred = rf_classifier.predict(X_train)
y_val_pred = rf_classifier.predict(X_val)

# Calculate accuracy on training and validation sets
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the accuracies
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

# Check if overfitting
if train_accuracy > val_accuracy:
    print("The model is overfitting.")
else:
    print("The model is not overfitting.")

NameError: name 'train_test_split' is not defined

In [46]:
model = pd.DataFrame([['Random Forest Baseline', train_accuracy,val_accuracy]],
               columns = ['Model', 'Train accuracy', 'Validation accuracy'])
model_result = pd.concat([model_result,model],axis=0,ignore_index = True)
model_result

NameError: name 'train_accuracy' is not defined

#**Gradient Boosting Classifier**

In [49]:
from sklearn.impute import SimpleImputer

# Initialize the imputer to fill NaN values with the mean of the column
imputer = SimpleImputer(strategy='mean')

# Fit the imputer and transform the training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Now fit the model with the imputed data
gb_classifier.fit(X_train_imputed, y_train)

# Make predictions and evaluate
y_pred_gb = gb_classifier.predict(X_test_imputed)
accuracy = gb_classifier.score(X_test_imputed, y_test)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(y_test, y_pred_gb)
print("\nClassification Report:\n", report)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_gb)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


NameError: name 'X_train' is not defined

In [51]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# Initialize the imputer to fill NaN values with the mean of the column
imputer = SimpleImputer(strategy='mean')

# Impute missing values in the training and validation sets
X_imputed = imputer.fit_transform(X)

# Split the imputed data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Initialize the GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Train the model on the training set
gb_classifier.fit(X_train, y_train)

# Predictions on training and validation sets
y_train_pred = gb_classifier.predict(X_train)
y_val_pred = gb_classifier.predict(X_val)

# Calculate accuracy on training and validation sets
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the accuracies
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

# Check for overfitting
if train_accuracy > val_accuracy:
    print("The model is overfitting.")
else:
    print("The model is not overfitting.")


NameError: name 'X' is not defined

In [53]:
model = pd.DataFrame([['XGBoost Classifier Baseline', train_accuracy,val_accuracy]],
               columns = ['Model', 'Train accuracy', 'Validation accuracy'])
model_result = pd.concat([model_result,model],axis=0,ignore_index = True)
model_result

NameError: name 'train_accuracy' is not defined

#**ROC-AUC Curve**

In [56]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import pandas as pd

# Function to train and evaluate the model
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Evaluate the model
    report = classification_report(y_test, y_pred)

    # Print accuracy and classification report
    #print(f"\nAccuracy for {model_name}: {accuracy:.2f}")
    #print(f"Classification Report for {model_name}:\n{report}")

    # Return accuracy, classification report, and predictions
    return accuracy, report, y_pred

# Initialize models
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)
decision_tree = DecisionTreeClassifier(random_state=42)
logistic_regression = LogisticRegression(random_state=42)

# Initialize imputer to handle missing values
imputer = SimpleImputer(strategy='mean')

# Impute missing values for training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Create a dictionary to store the classification reports and predictions
model_reports = {}

# Train and evaluate each model
print("Now.... RF")
accuracy, report, y_pred_rf = train_and_evaluate_model(random_forest, X_train_imputed, y_train, X_test_imputed, y_test, 'Random Forest')
model_reports['Random Forest'] = report

print("Now.... GB")
accuracy, report, y_pred_gb = train_and_evaluate_model(gradient_boosting, X_train_imputed, y_train, X_test_imputed, y_test, 'Gradient Boosting')
model_reports['Gradient Boosting'] = report

print("Now.... DT")
accuracy, report, y_pred_dt = train_and_evaluate_model(decision_tree, X_train_imputed, y_train, X_test_imputed, y_test, 'Decision Tree')
model_reports['Decision Tree'] = report

print("Now.... Logistic Regression")
accuracy, report, y_pred_lr = train_and_evaluate_model(logistic_regression, X_train_imputed, y_train, X_test_imputed, y_test, 'Logistic Regression')
model_reports['Logistic Regression'] = report

# Save the classification reports to a CSV file
reports_df = pd.DataFrame.from_dict(model_reports, orient='index', columns=['Classification Report'])
reports_df.to_csv('classification_reports.csv')

# Binarize the output labels for ROC curve calculation (multiclass case)
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])  # Assuming three classes (0, 1, 2); modify if necessary

# Initialize lists to store fpr, tpr, and roc_auc for each model
fpr_list = []
tpr_list = []
roc_auc_list = []

# Compute ROC curve and ROC AUC for each model (for each class)
for y_pred in [y_pred_lr, y_pred_dt, y_pred_rf, y_pred_gb]:
    fpr = {}
    tpr = {}
    roc_auc = {}

    # For each class, calculate ROC curve and AUC
    for i in range(y_test_bin.shape[1]):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred == i)
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Store fpr, tpr, and roc_auc
    fpr_list.append(fpr)
    tpr_list.append(tpr)
    roc_auc_list.append(roc_auc)

# Plot ROC curves for each model
plt.figure(figsize=(10, 8))
for i, roc_auc in enumerate(roc_auc_list):
    for j in roc_auc:
        plt.plot(fpr_list[i][j], tpr_list[i][j], lw=2, label=f'Model {i+1} Class {j} (AUC = {roc_auc[j]:0.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()


NameError: name 'RandomForestClassifier' is not defined

#**Benchmark Model Selection and Hyperparameter Tuning**

**Logistic Regression is our Benchmark model**

In [59]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Handle missing values by imputing with the mean value
imputer = SimpleImputer(strategy='median')

# Impute the training and testing data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Now define the hyperparameters for tuning using GridSearchCV
param_grid = {
    'penalty': ['l1', 'l2'],  # Regularization penalty
    'C': [0.001, 0.01, 0.1, 1, 4, 100],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga']  # Algorithm to use in the optimization problem
}

# Initialize Logistic Regression Classifier
lr_classifier = LogisticRegression(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(lr_classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the GridSearchCV to the imputed training data
grid_search.fit(X_train_imputed, y_train)

# Print the best hyperparameters found
print("Best Hyperparameters:", grid_search.best_params_)

# Use the best model found by GridSearchCV
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the imputed test data using the best model
y_pred_lr_tuned = best_lr_classifier.predict(X_test_imputed)

# Calculate and print accuracy using the best model
accuracy_tuned = best_lr_classifier.score(X_test_imputed, y_test)
print("Tuned Model Accuracy:", accuracy_tuned)

# Generate a classification report using the tuned model
report_tuned = classification_report(y_test, y_pred_lr_tuned)
print("\nTuned Model Classification Report:\n", report_tuned)

# Calculate confusion matrix using the tuned model
cm_tuned = confusion_matrix(y_test, y_pred_lr_tuned)

# Plot confusion matrix using the tuned model
plt.figure(figsize=(8, 6))
sns.heatmap(cm_tuned, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix (Tuned Model)')
plt.show()


NameError: name 'X_train' is not defined

In [61]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle missing values by imputing with the mean value
imputer = SimpleImputer(strategy='median')

# Impute the training and validation data
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

# Train the model on the imputed training set
best_lr_classifier.fit(X_train_imputed, y_train)

# Predictions on the imputed training and validation sets
y_train_pred = best_lr_classifier.predict(X_train_imputed)
y_val_pred = best_lr_classifier.predict(X_val_imputed)

# Calculate accuracy on the training and validation sets
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the accuracies
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)


NameError: name 'X' is not defined

#**Pickle the Benchmark model**

In [64]:
import pickle

# Pickle the tuned logistic regression model
with open('tuned_logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(best_lr_classifier, file)


NameError: name 'best_lr_classifier' is not defined

In [66]:
# Pickle the Scaler model
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

NameError: name 'scaler' is not defined

In [68]:
# Pickle the PCA model
with open('pca.pkl', 'wb') as file:
    pickle.dump(pca, file)

NameError: name 'pca' is not defined